In [1]:
import pandas as pd

In [11]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [13]:
import matplotlib.pyplot as plt

In [1]:
import yfinance as yf

In [6]:
df = pd.read_csv('Users/ericvincent/Downloads/RWTCd/Data1-Table1.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Users/ericvincent/Downloads/RWTCd/Data1-Table1.csv'

In [3]:
def load_csv(tickers: list, start:str, end:str):
    """
    Creating a function that would allow us to load a csv containing the Adj close, High, Low and volume of relevant tickers
    tickers should be a list
    """
    csv = yf.download(tickers=tickers, start=start, end=end)
    csv.drop(columns='Close', inplace=True)
    csv = csv[1:].copy()

    return csv

In [6]:
df = load_csv('MSFT', '2000-01-01', '2022-11-27')

[*********************100%***********************]  1 of 1 completed


In [7]:
df

,Open,High,Low,Adj Close,Volume
Date,,,,,
2000-01-04,56.781250,58.562500,56.125000,35.221275,54119000
2000-01-05,55.562500,58.187500,54.687500,35.592644,64059600
2000-01-06,56.093750,56.937500,54.187500,34.400349,54976600
2000-01-07,54.312500,56.125000,53.656250,34.849899,62013600
2000-01-10,56.718750,56.843750,55.687500,35.103996,44963600
...,...,...,...,...,...
2022-11-18,243.509995,243.740005,239.029999,241.220001,27591800
2022-11-21,241.429993,244.669998,241.190002,242.050003,26394700
2022-11-22,243.589996,245.309998,240.710007,245.029999,19665700


In [11]:
df['intraday_move'] = (df.High / df.Low)-1

In [16]:
df.pct_change()

,Open,High,Low,Adj Close,Volume,intraday_move
Date,,,,,,
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,-0.021464,-0.006403,-0.025612,0.010544,0.183680,0.473641
2000-01-06,0.009561,-0.021482,-0.009143,-0.033498,-0.141790,-0.207036
2000-01-07,-0.031755,-0.014270,-0.009804,0.013068,0.128000,-0.093384
2000-01-10,0.044304,0.012806,0.037857,0.007291,-0.274940,-0.548729
...,...,...,...,...,...,...
2022-11-18,0.024098,0.002014,0.005891,-0.001903,0.193236,-0.166828
2022-11-21,-0.008542,0.003816,0.009037,0.003441,-0.043386,-0.267765
2022-11-22,0.008947,0.002616,-0.001990,0.012311,-0.254938,0.324474


In [17]:
df

,Open,High,Low,Adj Close,Volume,intraday_move
Date,,,,,,
2000-01-04,56.781250,58.562500,56.125000,35.221275,54119000,0.043430
2000-01-05,55.562500,58.187500,54.687500,35.592644,64059600,0.064000
2000-01-06,56.093750,56.937500,54.187500,34.400349,54976600,0.050750
2000-01-07,54.312500,56.125000,53.656250,34.849899,62013600,0.046010
2000-01-10,56.718750,56.843750,55.687500,35.103996,44963600,0.020763
...,...,...,...,...,...,...
2022-11-18,243.509995,243.740005,239.029999,241.220001,27591800,0.019705
2022-11-21,241.429993,244.669998,241.190002,242.050003,26394700,0.014428
2022-11-22,243.589996,245.309998,240.710007,245.029999,19665700,0.019110


In [18]:
df['daily_return'] = df['Adj Close'].pct_change()

In [20]:
tickers = ['AAPL',
 'MSFT',
 'GOOG',
 'AMZN',
 'BRK.B',
 'TSLA',
 'UNH',
 'XOM',
 'JNJ',
 'WMT',
 'NVDA',
 'JPM',
 'V',
 'CVX',
 'PG',
 'LLY',
 'MA',
 'HD',
 'META',
 'BAC',
 'ABBV',
 'PFE',
 'KO',
 'MRK',
 'PEP',
 'COST',
 'ORCL',
 'AVGO',
 'TMO',
 'MCD',
 'CSCO',
 'ACN',
 'DHR',
 'TMUS',
 'ABT',
 'WFC',
 'DIS',
 'LIN',
 'NEE',
 'BMY',
 'NKE',
 'VZ',
 'TXN',
 'UPS',
 'COP',
 'ADBE',
 'CMCSA',
 'CRM',
 'PM',
 'MS',
 'AMGN',
 'SCHW',
 'HON',
 'RTX',
 'QCOM',
 'T',
 'IBM',
 'DE',
 'CVS',
 'LOW',
 'GS',
 'UNP',
 'NFLX',
 'LMT',
 'CAT',
 'AMD',
 'INTC',
 'ELV',
 'SPGI',
 'AXP',
 'SBUX',
 'INTU',
 'BLK',
 'ADP',
 'GILD',
 'PLD',
 'MDT',
 'BA',
 'AMT',
 'CI',
 'GE',
 'TJX',
 'ISRG',
 'C',
 'AMAT',
 'PYPL',
 'MDLZ',
 'CB',
 'SYK',
 'ADI',
 'MMC',
 'EOG',
 'NOW',
 'VRTX',
 'MO',
 'NOC',
 'EL',
 'REGN',
 'PGR',
 'BKNG',
 'DUK',
 'TGT',
 'SLB',
 'SO',
 'MMM',
 'ITW',
 'ZTS',
 'GD',
 'APD',
 'HUM',
 'MRNA',
 'BDX',
 'CSX',
 'WM',
 'PNC',
 'HCA',
 'ETN',
 'USB',
 'FISV',
 'SHW',
 'OXY',
 'CL',
 'MU',
 'CME',
 'AON',
 'LRCX',
 'BSX',
 'EQIX',
 'TFC',
 'PXD',
 'CHTR',
 'CCI',
 'MET',
 'ATVI',
 'ICE',
 'MPC',
 'NSC',
 'DG',
 'GM',
 'EMR',
 'F',
 'KLAC',
 'MCO',
 'FCX',
 'KDP',
 'MNST',
 'MCK',
 'VLO',
 'ORLY',
 'ADM',
 'PSX',
 'PSA',
 'SRE',
 'SNPS',
 'MAR',
 'D',
 'GIS',
 'AEP',
 'AZO',
 'KHC',
 'APH',
 'HSY',
 'CNC',
 'CTVA',
 'EW',
 'CTAS',
 'A',
 'ROP',
 'JCI',
 'CDNS',
 'FDX',
 'NXPI',
 'AIG',
 'KMB',
 'AFL',
 'HES',
 'MSI',
 'PAYX',
 'DVN',
 'TRV',
 'BIIB',
 'DXCM',
 'SYY',
 'LHX',
 'RSG',
 'ENPH',
 'ECL',
 'ADSK',
 'MCHP',
 'ANET',
 'KMI',
 'CMG',
 'FTNT',
 'AJG',
 'STZ',
 'TT',
 'WMB',
 'MSCI',
 'O',
 'IQV',
 'TEL',
 'ROST',
 'PRU',
 'EXC',
 'PH',
 'FIS',
 'SPG',
 'COF',
 'NUE',
 'XEL',
 'HLT',
 'CARR',
 'PCAR',
 'BK',
 'NEM',
 'DOW',
 'EA',
 'WBA',
 'DD',
 'ALL',
 'YUM',
 'AMP',
 'CMI',
 'ILMN',
 'BF.B',
 'TDG',
 'IDXX',
 'ED',
 'KR',
 'ABC',
 'DLTR',
 'RMD',
 'ALB',
 'HAL',
 'NDAQ',
 'LVS',
 'ODFL',
 'WELL',
 'AME',
 'CSGP',
 'OTIS',
 'MTD',
 'SBAC',
 'ON',
 'VICI',
 'DLR',
 'CEG',
 'KEYS',
 'PPG',
 'WEC',
 'CTSH',
 'ROK',
 'GWW',
 'PCG',
 'HPQ',
 'FAST',
 'DFS',
 'MTB',
 'PEG',
 'OKE',
 'DHI',
 'APTV',
 'BKR',
 'GLW',
 'LYB',
 'ES',
 'BAX',
 'STT',
 'VRSK',
 'TROW',
 'WBD',
 'AWK',
 'IT',
 'GPN',
 'HRL',
 'FANG',
 'WTW',
 'RJF',
 'GPC',
 'IFF',
 'CDW',
 'TSCO',
 'FITB',
 'ARE',
 'URI',
 'ZBH',
 'K',
 'LEN',
 'EBAY',
 'EIX',
 'CBRE',
 'EFX',
 'VMC',
 'TSN',
 'HIG',
 'FTV',
 'WY',
 'EQR',
 'AVB',
 'MKC',
 'ETR',
 'LUV',
 'ULTA',
 'AEE',
 'MLM',
 'FE',
 'PFG',
 'FRC',
 'DTE',
 'DAL',
 'HBAN',
 'IR',
 'CTRA',
 'ANSS',
 'ACGL',
 'PPL',
 'RF',
 'VRSN',
 'LH',
 'EXR',
 'PWR',
 'CF',
 'CAH',
 'CFG',
 'XYL',
 'HPE',
 'EPAM',
 'DOV',
 'WAT',
 'WRB',
 'TDY',
 'PAYC',
 'ROL',
 'NTRS',
 'MRO',
 'CNP',
 'INVH',
 'CHD',
 'AES',
 'MOH',
 'JBHT',
 'MAA',
 'BBY',
 'CLX',
 'HOLX',
 'WAB',
 'DRI',
 'EXPD',
 'STE',
 'AMCR',
 'VTR',
 'IEX',
 'CAG',
 'CMS',
 'KEY',
 'MPWR',
 'BALL',
 'J',
 'BR',
 'GRMN',
 'PKI',
 'TTWO',
 'INCY',
 'FDS',
 'MOS',
 'SEDG',
 'CINF',
 'ABMD',
 'DGX',
 'WST',
 'ATO',
 'TRGP',
 'BRO',
 'SYF',
 'FOX',
 'FOXA',
 'NTAP',
 'FMC',
 'EQT',
 'OMC',
 'SJM',
 'LYV',
 'CPB',
 'HWM',
 'CPRT',
 'AVY',
 'IRM',
 'COO',
 'ALGN',
 'SWKS',
 'EXPE',
 'RCL',
 'ETSY',
 'APA',
 'GEN',
 'TXT',
 'LDOS',
 'LKQ',
 'TER',
 'PTC',
 'TRMB',
 'AKAM',
 'NVR',
 'UAL',
 'LNT',
 'FLT',
 'KIM',
 'ZBRA',
 'TYL',
 'DPZ',
 'JKHY',
 'MGM',
 'ESS',
 'L',
 'PEAK',
 'MTCH',
 'NDSN',
 'EVRG',
 'VTRS',
 'IPG',
 'BEN',
 'CBOE',
 'TECH',
 'SIVB',
 'VFC',
 'IP',
 'HST',
 'UDR',
 'POOL',
 'RE',
 'PARA',
 'SNA',
 'CPT',
 'LW',
 'PKG',
 'CRL',
 'SWK',
 'BIO',
 'WDC',
 'CHRW',
 'STX',
 'MAS',
 'GL',
 'CE',
 'REG',
 'NI',
 'BXP',
 'HSIC',
 'CCL',
 'TFX',
 'CZR',
 'NWS',
 'NWSA',
 'KMX',
 'EMN',
 'JNPR',
 'PHM',
 'CDAY',
 'ALLE',
 'QRVO',
 'BWA',
 'NRG',
 'MKTX',
 'WRK',
 'UHS',
 'FFIV',
 'AOS',
 'CMA',
 'AAL',
 'BBWI',
 'HII',
 'AAP',
 'TPR',
 'FRT',
 'IVZ',
 'PNW',
 'HAS',
 'WYNN',
 'FBHS',
 'SBNY',
 'DISH',
 'RHI',
 'WHR',
 'ZION',
 'CTLT',
 'PNR',
 'SEE',
 'RL',
 'NCLH',
 'DXC',
 'GNRC',
 'AIZ',
 'XRAY',
 'LNC',
 'DVA',
 'MHK',
 'OGN',
 'LUMN',
 'ALK',
 'NWL',
 'VNO',
 'TAP']


In [22]:
df = load_csv(tickers, '2000-01-01', '2022-11-27')

[*********************100%***********************]  502 of 502 completed

5 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- CI: Error occurred while retrieving timeseries from Redis, keys: [RedisKey [key=CI, cluster=finance]]
- AIZ: Error occurred while retrieving timeseries from Redis, keys: [RedisKey [key=AIZ, cluster=finance]]
- DPZ: Error occurred while retrieving timeseries from Redis, keys: [RedisKey [key=DPZ, cluster=finance]]
- BRK.B: No data found for this date range, symbol may be delisted
[*********************100%***********************]  503 of 502 completed

In [23]:
df

Adj Close                                                         \
                     A    AAL         AAP        AAPL        ABBV         ABC   
Date                                                                            
2000-01-04   40.907444    NaN         NaN    0.786965         NaN    2.819288   
2000-01-05   38.369949    NaN         NaN    0.798481         NaN    3.038026   
2000-01-06   36.908974    NaN         NaN    0.729382         NaN    3.268915   
2000-01-07   39.984726    NaN         NaN    0.763932         NaN    3.694239   
2000-01-10   42.406872    NaN         NaN    0.750496         NaN    3.791455   
...                ...    ...         ...         ...         ...         ...   
2022-11-18  146.190002  14.05  147.500000  151.289993  154.979996  162.000000   
2022-11-21  145.139999  13.85  148.240005  148.009995  157.110001  160.300003   
2022-11-22  156.860001  13.98  149.550003  150.179993  159.789993  164.710007   
2022-11-23  155.350006  14.42  150.550003  151.070007  159.389999  165.529999   
2022-11-25  156.960007  14.50  150.000000  148.110001  159.619995  167.699997   

                                                           ...     Volume  \
                  ABMD         ABT       ACGL         ACN  ...       WYNN   
Date                                                       ...              
2000-01-04   17.812500    9.116958   1.270833         NaN  ...        NaN   
2000-01-05   18.000000    9.100201   1.388889         NaN  ...        NaN   
2000-01-06   18.031250    9.418623   1.375000         NaN  ...        NaN   
2000-01-07   17.937500    9.519175   1.451389         NaN  ...        NaN   
2000-01-10   20.500000    9.452140   1.500000         NaN  ...        NaN   
...                ...         ...        ...         ...  ...        ...   
2022-11-18  377.290009  103.870003  57.669998  286.500000  ...  1930800.0   
2022-11-21  376.619995  103.879997  57.160000  287.019989  ...  2094800.0   
2022-11-22  378.000000  104.870003  57.779999  292.890015  ...  1374800.0   
2022-11-23  377.779999  106.019997  58.049999  294.529999  ...  1087700.0   
2022-11-25  378.079987  106.959999  58.730000  296.399994  ...  1311600.0   

                                                                              \
                XEL       XOM     XRAY       XYL      YUM        ZBH    ZBRA   
Date                                                                           
2000-01-04   425200  14510800   317700       NaN  3315031        NaN  522450   
2000-01-05   500200  17485000  1188000       NaN  4642602        NaN  612225   
2000-01-06   344100  19461600   534300       NaN  3947658        NaN  263925   
2000-01-07   469500  16603800  1401000       NaN  6063647        NaN  333900   
2000-01-10   522000   8545400   613200       NaN  5531172        NaN  325800   
...             ...       ...      ...       ...      ...        ...     ...   
2022-11-18  2922000  15407000  1962700  637400.0  2985400   770500.0  504500   
2022-11-21  2876400  20973000  1761700  529400.0  1621000   655400.0  402700   
2022-11-22  2625200  17719100  2033200  509900.0  1507000   834200.0  442100   
2022-11-23  1947100  11546700  1435400  585600.0  1881000  1243100.0  464600   
2022-11-25  1193300   7136000   614300  539600.0   870600   427900.0  202400   

                                
               ZION        ZTS  
Date                            
2000-01-04   816100        NaN  
2000-01-05  1124700        NaN  
2000-01-06  1112100        NaN  
2000-01-07   782000        NaN  
2000-01-10   408800        NaN  
...             ...        ...  
2022-11-18  1843500  2410500.0  
2022-11-21  1450500  1998900.0  
2022-11-22   981900  1628500.0  
2022-11-23   862600  1445900.0  
2022-11-25   309400   889500.0  

[5762 rows x 2510 columns]